In [9]:
import os
import shutil
import re
import pandas as pd
from datetime import datetime
import log_config as lc

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [10]:
import pandas as pd

In [11]:
if os.name == 'posix':
    file_loc = r"/home/shashankraj/Documents/DATA_Dump"
    log_file = file_loc+"/"+"Logs"
    final_files = file_loc+"/"+"Final_Df"
    log_file = file_loc+"/"+"Logs"
    backup = file_loc+"/"+"Backup"
    #today = str(datetime.today().date())
    trigger_file = file_loc+"/file_trigger/"+"new_data_received.txt"
else:
    file_loc=r"D:\One Drive Anza\OneDrive - Anza Services LLP\DATA_Dump\Final_Df"
    log_file = file_loc+"\\"+"Logs"
    log_file = file_loc+"\\"+"Logs"
    final_files = file_loc+"\\"+"Final_Df"
    backup = file_loc+"\\"+"Backup"
    #today = str(datetime.today().date())
    trigger_file = file_loc+"\\file_trigger\\"+"new_data_received.txt"

In [12]:
skip_rows = {'Client Billing Descending': 0, "Fee Breakdown by Dept and Fee Earner": 3,
             "Fee Summary by Dept and Fee Earner": 3, "Fees Billed": 3, "Matter Source of Business inc Matter Bills": 0,
             "Matters Opened by FE": 3, "Payment Received Analysis": 3, "Total Hours by Fee Earner-With Billings": 0}


In [13]:
def remove_cols(df):
    """ 
        This will remove all the columns that contain the word Textbox in them. 
        This Function takes a DataFrame as in input and returns all the columns except TextBox. 
    """

    cols = df.columns
    new_cols = []
    txt_chk = re.compile(r'Textbox')
    tot_hrs_col_name = ["RecordedHours2","NonChargeHours2","WOHours2","TotalHour2","bankRef"]
    new_cols = [col_name for col_name in cols if not(txt_chk.search(col_name)) and col_name not in tot_hrs_col_name]
    return new_cols

In [14]:
def get_rows(dct, match):
    """
        Takes a Dictionary and a filename as inputs and Returns how many rows need to be skipped for a filename. 
        Returns the Number of rows to skip, while creating a DataFrame.
    """
    for val in dct.keys():
        if re.match(val, match):
            return dct[val]

In [15]:
def get_date_from_Filename(fname):
    """
        Accepts a Filename that has fname_date.csv format. 
        It Extracts the From Date form the File and Returns the same. 
        These Are Datetime Objects.  
        If the Filename has only start date, it will just return the same date for Both Start and End Date. 
    """

    pattern = re.compile(r'_\d*')
    match = pattern.findall(fname)
    dt = match[0]
    dt = dt.split("_")[1]
    file_date = pd.to_datetime(dt, format='%d%m%Y')
    return file_date

In [16]:
def categorize_files(file_loc):
    import sys
    
    if os.name == 'posix':
            os.system('clear')
    else:
        os.system('cls')

    log_loc = file_loc + "/" + "Logs"
    unprocessed = file_loc + "/Unprocessed"
    processed = file_loc + "/Processed"
    ess_fold_list = [log_loc,unprocessed,processed]

    cat_file_logger = lc.start_log(log_loc)
    
    for fold in ess_fold_list:
        if os.path.exists(fold):
            cat_file_logger.info(f'\n[{fold}] already Exists in [{file_loc}]\n')
        else:
            os.mkdir(file_loc+"/"+fold)
            cat_file_logger.info(f'\nCreating Folder {fold} in {file_loc}\n')

    total_csv = len([f for f in os.listdir(file_loc) if f.endswith('.csv')])
    if total_csv > 0:
        print(f"Found {total_csv} csv files")
    elif total_csv == 0:
        print("No CSV Files Found. Exiting.")
        sys.exit()

    process_files = {}
    discard_files = {}
    file_list = os.listdir(file_loc)

    cat_file_logger.info(f"Any xls file and files having Pie in the names will not be Processed")

    # Segregating the Files.
    discard_files['all_pie'] = [files for files in file_list if len(re.compile(r'[\sa-zA-Z\s]+Pie \w+_\d+.csv').findall(files))]
    discard_files['all_xlsx'] = [files for files in file_list if files.endswith(".xlsx")]

    # Move the above files to Unprocessed Folder before moving ahead
    for f in discard_files.keys():
                cat_file_logger.info(f'Moving out files of list [{f}] to folder [{unprocessed}]')
                try:
                    [shutil.move(file_loc + "/" + file, unprocessed)  for file in discard_files[f]]
                except:
                    cat_file_logger.error("File Already exists in Destination")
                else:
                    cat_file_logger.info("Files moved")


    process_files['client_billing'] = [files for files in file_list if len(re.compile(r'Client [a-zA-Z\s]+_\d+.csv').findall(files))]
    cnt = len(process_files['client_billing'])
    cat_file_logger.info(f'Found [{cnt}] files of Client BIlling ')
    
    process_files['fee_brkdn_dept_fe'] = [files for files in file_list if len(re.compile(r'Fee Breakdown [a-zA-Z\s]+_\d+.csv').findall(files))]
    cnt = len(process_files['fee_brkdn_dept_fe'])
    cat_file_logger.info(f'Found [{cnt}] files of Fee Breakdown by Dept ')
    
    process_files['fee_summ_dept_fe'] = [files for files in file_list if len(re.compile(r'Fee Summary [a-zA-Z\s]+_\d+.csv').findall(files))]
    cnt = len(process_files['fee_summ_dept_fe'])
    cat_file_logger.info(f'Found [{cnt}] files of Fee Summary by Dept ')
        
    process_files['fees_billed'] = [files for files in file_list if len(re.compile(r'Fees B[a-zA-Z\s]+_\d+.csv').findall(files))]
    cnt = len(process_files['fees_billed'])
    cat_file_logger.info(f'Found [{cnt}] files of Fees BIlled ')
        
    process_files['matter_src'] = [files for files in file_list if len(re.compile(r'Matter Source [a-zA-Z\s()]+_\d+.csv').findall(files))]
    cnt = len(process_files['matter_src'])
    cat_file_logger.info(f'Found [{cnt}] files of Matter Source Reference ')
     
    process_files['matter_opened'] = [files for files in file_list if len(re.compile(r'Matters Open[\sa-zA-Z\s()]+_\d+.csv').findall(files))]
    cnt = len(process_files['matter_opened'])
    cat_file_logger.info(f'Found [{cnt}] files of Matter Opened by FE ')
    
    process_files['payment_rcv'] = [files for files in file_list if len(re.compile(r'Payment [\sa-zA-Z\s()]+_\d+.csv').findall(files))]
    cnt = len(process_files['payment_rcv'])
    cat_file_logger.info(f'Found [{cnt}] files of Payment Received ')
       
    process_files['tot_hrs_fe'] = [files for files in file_list if len(re.compile(r'([tT]otal[\sa-z-A-Z\s]*_\d+.csv)').findall(files))]
    cnt = len(process_files['tot_hrs_fe'])
    cat_file_logger.info(f'Found [{cnt}] files of Total Hours by Fee Earner ')

    for f in process_files.keys():
        cat_file_logger.info(f'Moving category [{f}]')
        final_file = file_loc + "/" + file
        try:
            [shutil.move(final_file, processed)  for file in process_files[f]]
            #[cat_file_logger.info(f' Moving files {file} to processed')  for file in process_files[f]]
        except:
            [shutil.move(final_file, processed) if os.path.exists(file_loc + "/" + file) else os.remove(processed+"/"+final_file) & shutil.move(final_file, processed) for file in process_files[f]]
            cat_file_logger.info("File already Existed, Moving Latest version of [{}]".format(f))
        else:
            [cat_file_logger.info(f'Moving File --> {file}')  for file in process_files[f]]
            pass
        
    return process_files

In [17]:
p_list = (1,2,3,4,5,6,7,8,9)

[x if x%2 == 0 else "Prime" for x in p_list]

['Prime', 2, 'Prime', 4, 'Prime', 6, 'Prime', 8, 'Prime']

In [18]:
def categorize_files2(file_loc):

    import sys
    
    if os.name == 'posix':
            os.system('clear')
    else:
        os.system('cls')

    log_loc = file_loc + "/Logs"
    cat_file_logger = lc.start_log(log_loc)
    unprocessed = file_loc + "/Unprocessed"
    processed = file_loc + "/Processed"

    ess_fold_list = [log_loc,unprocessed,processed]

    for fold in ess_fold_list:
        if os.path.exists(fold):
            cat_file_logger.info(f'\n[{fold}] already Exists in [{file_loc}]\n')
        else:
            os.mkdir(file_loc+"/"+fold)
            cat_file_logger.info(f'\nCreating Folder {fold} in {file_loc}\n')

    total_csv = len([f for f in os.listdir(file_loc) if f.endswith('.csv')])
    if total_csv > 0:
        cat_file_logger.info(f"Found {total_csv} csv files")
    elif total_csv == 0:
        cat_file_logger.error("No CSV Files Found. Exiting.")
        sys.exit()

    process_files = {}
    discard_files = {}
    file_list = os.listdir(file_loc)

    cat_file_logger.info(f"Any xls file and files having Pie in the names will not be Processed")

    # Segregating the Files.
    discard_files['all_pie'] = [files for files in file_list if len(re.compile(r'[\sa-zA-Z\s]+Pie \w+_\d+.csv').findall(files))]
    discard_files['all_xlsx'] = [files for files in file_list if files.endswith(".xlsx")]

    # Move the above files to Unprocessed Folder before moving ahead
    for f in discard_files.keys():
                cat_file_logger.info(f'Moving out files of list [{f}] to folder [{unprocessed}]')
                # [shutil.move(file_loc + "/" + file, unprocessed)  for file in discard_files[f]]
                [cat_file_logger.info(f"Moving File {file} to  unprocessed")  for file in discard_files[f]]

    process_files['client_billing'] = [files for files in file_list if len(re.compile(r'Client [a-zA-Z\s]+_\d+.csv').findall(files))]
    cnt = len(process_files['client_billing'])
    cat_file_logger.info(f'Found [{cnt}] files of Client BIlling ')
    
    process_files['fee_brkdn_dept_fe'] = [files for files in file_list if len(re.compile(r'Fee Breakdown [a-zA-Z\s]+_\d+.csv').findall(files))]
    cnt = len(process_files['fee_brkdn_dept_fe'])
    cat_file_logger.info(f'Found [{cnt}] files of Fee Breakdown by Dept ')
    
    process_files['fee_summ_dept_fe'] = [files for files in file_list if len(re.compile(r'Fee Summary [a-zA-Z\s]+_\d+.csv').findall(files))]
    cnt = len(process_files['fee_summ_dept_fe'])
    cat_file_logger.info(f'Found [{cnt}] files of Fee Summary by Dept ')
        
    process_files['fees_billed'] = [files for files in file_list if len(re.compile(r'Fees B[a-zA-Z\s]+_\d+.csv').findall(files))]
    cnt = len(process_files['fees_billed'])
    cat_file_logger.info(f'Found [{cnt}] files of Fees BIlled ')
        
    process_files['matter_src'] = [files for files in file_list if len(re.compile(r'Matter Source [a-zA-Z\s()]+_\d+.csv').findall(files))]
    cnt = len(process_files['matter_src'])
    cat_file_logger.info(f'Found [{cnt}] files of Matter Source Reference ')
     
    process_files['matter_opened'] = [files for files in file_list if len(re.compile(r'Matters Open[\sa-zA-Z\s()]+_\d+.csv').findall(files))]
    cnt = len(process_files['matter_opened'])
    cat_file_logger.info(f'Found [{cnt}] files of Matter Opened by FE ')
    
    process_files['payment_rcv'] = [files for files in file_list if len(re.compile(r'Payment [\sa-zA-Z\s()]+_\d+.csv').findall(files))]
    cnt = len(process_files['payment_rcv'])
    cat_file_logger.info(f'Found [{cnt}] files of Payment Received ')
       
    process_files['tot_hrs_fe'] = [files for files in file_list if len(re.compile(r'([tT]otal[\sa-z-A-Z\s]*_\d+.csv)').findall(files))]
    cnt = len(process_files['tot_hrs_fe'])
    cat_file_logger.info(f'Found [{cnt}] files of Total Hours by Fee Earner ')

    for f in process_files.keys():
        cat_file_logger.info(f'Moving category [{f}]')
        try:
            #[shutil.move(file_loc + "/" + file, processed)  for file in process_files[f]]
            [cat_file_logger.info(f' Moving files {file} to processed')  for file in process_files[f]]
        except:
            cat_file_logger.info("Error Moving File {}".format(f))
        else:
            #[cat_file_logger.info(f'Moving File --> {file}')  for file in process_files[f]]
            pass
        
    return process_files

In [19]:
def concat_files(dict_list, file_loc, logfile_loc):
    concat_logger = lc.start_log(logfile_loc)
    df_all_files = {}
    dict_fname= ""
     # If a Filename is not in this Dictionary, then it will not be Considered. 
    # date = (datetime.now()).strftime("%m-%d-%y")
    concat_logger.info(f'Following Keys will be processed - [{dict_list.keys()}]')
    for file_cat in dict_list.keys():
        df_final = pd.DataFrame()
        concat_logger.info('*' * 50)
        concat_logger.info(f'Processing Category {file_cat}')
        for file in dict_list[file_cat]:
            dict_fname = file.split("_")[0]
            dfc_file = pd.read_csv((file_loc + "/Processed/" + file), skiprows=get_rows(skip_rows, file))
            dfc_file = dfc_file[remove_cols(dfc_file)]
            processing_date = get_date_from_Filename(file)
            dfc_file["Date_Added"] = processing_date
            df_final = pd.concat([df_final, dfc_file], ignore_index=True)
            df_final.fillna(0)
            df_final = df_final.replace(re.compile(r'£'), "").replace(re.compile(r','), "").replace(re.compile(r'\('),"-").replace(re.compile(r'\)'), "")

            for cols in df_final.columns:
                try:
                    df_final[cols].astype(float)
                except:
                    continue
                    # concat_logger.info(f'Skipping Column {cols}')
                else:
                    # concat_logger.info(f'Converting {cols} to float')
                    df_final[cols] = df_final[cols].astype(float)

        df_all_files[dict_fname] = df_final.sort_values(by="Date_Added",ascending=True)

    for f in df_all_files.keys():
        rows = df_all_files[f].shape[0]
        concat_logger.info(f'Will Add -> {rows} entries for [{f}] to the database')

    return df_all_files

In [21]:
dict_list = categorize_files2(file_loc)
all_files = concat_files(dict_list, file_loc, log_file)

KeyError: 'Date_Added'

In [ ]:
for file in all_files.keys():
        all_files[file].columns = [cols.lower() for cols in all_files[file].columns]
        fname = final_files+"/"+file+".csv"
        if os.path.exists(fname):
            print(f'{fname} already exists')
            all_files[file].to_csv(final_files+"/"+file+".csv",header=False,mode="a",index=False)
        else:
            print(f'Creating DF {fname}')
            all_files[file].to_csv(final_files+"/"+file+".csv",mode="a",index=False)
        all_files[file].to_parquet(backup+"/"+file+".parquet.gzip",compression = 'gzip')  
        print(f'Processed File [{file}]')

In [ ]:
all_files['Client Billing Descending'].head(5)

In [ ]:
tmp_df = {}
to_add = {}
to_update = {}

for file in all_files.keys():
    print(file)
    #tmp_df[file] = pd.DataFrame()

In [ ]:
for csv_file in os.listdir(final_files):
    if csv_file.endswith('.csv'):
        fname = csv_file.split(".")[0]
        tmp_df[fname] = pd.read_csv(final_files+"/"+csv_file)
        tmp_df[fname]['date_added'] = pd.to_datetime(tmp_df[fname]['date_added'])

tmp_df.keys()

In [ ]:
tmp_df['Client Billing Descending']

In [ ]:
all_files['Client Billing Descending']

In [ ]:
df = pd.DataFrame()

for file in all_files.keys():
    print(f'Processing {file}')
    for index,vals in all_files[file].iterrows():
        dt = pd.Timestamp(datetime.date(vals["Date_Added"]))
        if dt in tmp_df[file]['date_added'].values:
            print(f'Found {dt} in {vals} for {file}')
            pd.concat(vals,df)


In [ ]:
# new_val = {"client_surname1":["Newstone 40 Ducks Hill LLP","Cobley Ross"],
# "total_profit1":[2600,2590],
# "total_bill_amount":[3162,3192],
# "total_profit":[9000,8975],
# "total_bill_amount1":[10980,10990],
# "date_added":['2022-03-01','2022-03-01']}

# new_val
# df1 = pd.DataFrame(new_val)
# df1['date_added'] = pd.to_datetime(df1['date_added'])
# df1

# for index,vals in df1.iterrows():
#     clnt = vals["client_surname1"]
#     dt = pd.Timestamp(datetime.date(vals["date_added"]))
#     print(clnt,"--", dt)
#     if clnt and dt in df_cb.values:
#         print("Found")